<a href="https://colab.research.google.com/github/hsgw/keyboard-made-by-python/blob/main/notebook/en/pcb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> これは[キーボード #1 Advent Calendar 2022](https://adventar.org/calendars/7529)の20日目の記事の一部です。   
> トップページは[Pythonだけでキーボードを作る](https://5z6p.com/2022/12/21/ac2022/)です。

# Design PCB

<img src="../imgs/kbd_python_preview_top.png" height="300px">
<img src="../imgs/kbd_python_preview_bot.png" height="300px">

The goal of this notebook is to design the circuit for a keyboard and to get `Gerber files` for the production.
To make a pcb in the normal way, first draw a `circuit diagram` in the `circuit diagram editor` and output it as a `netlist` with the components to be used and their wiring information, and then physically place and wire the components on the `PCB` using the `PCB editor`.

## NOTE TO READ ARTICLE.
This notebook contains cells that will not work properly if run twice.

To prevent this, you should run `Runtime>Execute All Cells` from the menu above before starting to read the article. It will take some time for the execution to complete.

## Download resources
Run the following cell to clone the repository with kicad libraries and other resources.

In [ ]:
!git clone https://github.com/hsgw/keyboard-made-by-python/

# Design a netlist (schematic) with skidl
If you design with Python, you can define components and wiring information by code without drawing a schematic, and output a netlist directly.

[skidl](https://github.com/devbisme/skidl) uses the kicad component library as well as its own formats. The netlist is also compatible with the kicad and can be directly imported into pcbnew to make the pcb.

- skidl  https://github.com/devbisme/skidl
- skidl Document  https://devbisme.github.io/skidl/

## Install and import skidl
Install and import [skidl](https://github.com/devbisme/skidl).

In [ ]:
# Install will take some time
!pip install skidl

In [ ]:
# WARNING because the default path for kicad is not set in the environment variable
from skidl import *

## Declare constants
Declare values that appear repeatedly as constants. 

In [ ]:
KEY_COUNT = 10
COL_COUNT = 4
ROW_COUNT = 3
MATRIX_MAP = [
        (0,1),(0,2),(0,3),  
        (1,1),(1,2),(1,3),
  (2,0),(2,1),(2,2),(2,3)
]

## Loading kicad library and define parts
Add the path to the kicad library to be used.

If kicad is installed on your computer, the path to the default library has been added. In this environment, pass the path to the required libraries only.   
They are in `keyboard-made-by-python/hardware/kicad_libs` that has just been cloned from github.

In [ ]:
# resistor library paths
# require both symbol and footprint
lib_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs")
footprint_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs/kicad.pretty")

# If you use the same part more than one, define it as template with a symbol and footprint
diode = Part(
  "kicad_symbols", "D_Small_ALT", TEMPLATE, footprint="kicad:D_SOD123_hand"
)
switch = Part(
  "kicad_symbols",
  "SW_Push",
  TEMPLATE,
  footprint="kicad:SW_Cherry_MX_1.00u_PCB",
)

# define diodes and switches as array
diodes = diode(KEY_COUNT)
switches = switch(KEY_COUNT)

xiao = Part("kicad_symbols", "xiao_rp2040", footprint="kicad:xiao_rp2040")
oled = Part("kicad_symbols", "oled_i2c", footprint="kicad:oled_i2c")

# You can see the Pin information of the symbol when you print
print(diode, switch, xiao, oled)

## Connecting Pins and Net
Define a circuit by entering connections between the pins of the prepared components.

If possible, you should name the wiring itself with `Net`. By connecting the `Pins` of the components based on the `Net`, you can make the code easier to read by organizing the connections to the same place (power supply, switch matrix, etc.). It also serves as a good guide when importing a netlist into kicad, as it will be displayed there.

This design will also read the `pin numbers` from `Net` when wiring the PCB.

In [ ]:
# Viewing pin information is helpfull when wiring
print(xiao)

In [ ]:
# Make an array containing the netlist of ROW and COL of the switch matrix
netRows = [Net(f"ROW{i}") for i in range(ROW_COUNT)]
netCols = [Net(f"COL{i}") for i in range(COL_COUNT)]


# Connect ROW's Net -> switch's Pin 1 switch's Pin 2 -> diode's cathode, diode's anode -> COL's Net
# Net and Pin are connected by `&`
# Pin of a component can be accessed by part["pin name"]
# Pin of a part can be accessed by part["pin name"] 
# Two subscripts for in and out
# Example: Net or Pin connected to sw["1"] & sw["1 2"] & Net or Pin connected to sw["2"]
for sw, d, mapping in zip(switches, diodes, MATRIX_MAP):
  netRows[mapping[0]] & sw["1 2"] & d["K A"] & netCols[mapping[1]]

# Connect the switch matrix to xiao
# Add Pin to Net by `+`
netCols[0] += xiao[8]
netCols[1] += xiao[3]
netCols[2] += xiao[4]
netCols[3] += xiao[5]

netRows[0] += xiao[1]
netRows[1] += xiao[6]
netRows[2] += xiao[7]

# Connect oled and xiao
# You can connect by declaring Net directly like Net("3.3V")
Net("3.3V") & oled["Vcc"] & xiao["3.3V"]
Net("GND") & oled["GND"] & xiao["GND"]
Net("SDA") & oled["SDA"] & xiao[9]
Net("SCL") & oled["SCL"] & xiao[11]

# printing Net shows connected Pins
print(netRows[0])

## Run ERC and export netlist
Run ERC (Electrical Rule Check, schematic rule check) and export the netlist.
Now the schematic/netlist is complete!

In [ ]:
# You may get an error if you run other cells more than once
# In that case, restart and run all cells again
# You will get a warning about un-wired cells, but no problem
ERC()
generate_netlist(file_="keyboard.net")

The generated netlist can be imported by kicad pcbnew to create pcb as-is.

![imported by kicad pcbnew](https://github.com/hsgw/keyboard-made-by-python/blob/main/notebook/imgs/kicad_pcbnew.png?raw=1)

# Designing a PCB with pcbflow
Using [pcbflow](https://github.com/michaelgale/pcbflow), read the footprint and pin connections designed in skidl and physically place the components and wiring on the PCB. As needed, you can preview the image files and finally export a complete set of Gerber files for manufacturing.

The routing is based on the traditional [Turtle graphics](https://docs.python.org/ja/3/library/turtle.html)-like notations.

pcbflow is under development(?). and had some problems and unexpected behavior, so I have fixed and updated it. [folk](https://github.com/hsgw/pcbflow/tree/fix_kicad)

- pcbflow https://github.com/michaelgale/pcbflow
- folked pcbflow https://github.com/hsgw/pcbflow/tree/fix_kicad

## Install pcbflow
Execute the following cells to install.
Import and verify that there are no errors.

In [ ]:
# Takes a few minutes to install
!pip install git+https://github.com/hsgw/pcbflow/@fix_kicad

In [ ]:
from pcbflow import *

## Declare constants
As in the netlist, frequently used values are declared as constants.   
I also declare a function to convert the Y-axis of the coordinates because it was inverted and felt uncomfortable.

In [ ]:
BOARD_WIDTH = 76.0
BOARD_HEIGHT = 57.0

KEY_PITCH = 19.0

SCREW_HOLE = 2.2

LAYER_TOP = "GTL"
LAYER_BOTTOM = "GBL"

# Invert Y-axis
def pos(x, y):
  return (x, BOARD_HEIGHT - y)

## Declare variables for the PCB and circuit
Declare variables to access the PCB and the circuit.
The PCB is from pcbflow and the circuit is from skidl.

When declaring the `board`, specify the size of the outline and add an outline line.

In [ ]:
# from skidl
circuit = builtins.default_circuit
# for pcbflow
board = Board((BOARD_WIDTH, BOARD_HEIGHT))
board.add_outline()

## Configure design rules
Configure the design rules for the PCB, such as drill size, wire width, and so on.

Use these settings as the basis for routing the PCB.

In [ ]:
board.drc.trace_width = 0.5
board.drc.via_drill = 0.6
board.drc.via_annular_ring = 0.4
board.drc.clearance = 0.4

## Add drill holes
Make a hole to mount the PCB.

In [ ]:
board.add_hole(pos(KEY_PITCH * 2, KEY_PITCH), SCREW_HOLE)
board.add_hole(pos(KEY_PITCH * 3, KEY_PITCH), SCREW_HOLE)
board.add_hole(pos(KEY_PITCH * 3, KEY_PITCH * 2), SCREW_HOLE)
board.add_hole(pos(KEY_PITCH, KEY_PITCH * 2), SCREW_HOLE)

### Generate image and preview
Now let's generate image for preview

In [ ]:
from IPython.display import Image

# Export the current board as a png file
board.save_png("kbd_python", subdir="pcb_png")

# Show on output
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## Add position info to a part
Add position, rotation, and mounting surface info on the PCB to each of the skidl parts.

In this article, the position is specified immediately, but I repeatedly placed the parts on the PCB and previewed them to determine the position.

In [ ]:
# sw, d, xiao, oled are from skidl section

# Switches and diodes are positioned the same for every block
for sw, d, mapping in zip(switches, diodes, MATRIX_MAP):
  sw.pos = pos(
    mapping[1] * KEY_PITCH + KEY_PITCH / 2,
    mapping[0] * KEY_PITCH + KEY_PITCH / 2,
  )
  sw.side = "top"
  sw.rotate = 0
  d.pos = (sw.pos[0] + 6, sw.pos[1] + 5)
  d.side = "bottom"
  d.rotate = 0

xiao.pos = pos(11.25, 11.5)
xiao.side = "bottom"
xiao.rotate = 0

oled.pos = pos(9.5, 36)
oled.side = "top"
oled.rotate = 270

## Place the components on the PCB
Based on the location information added, place the parts in skidl while converting them to the PCB in pcbflow.

In [ ]:
for part in circuit.parts:
  try:
    # convert skidl parts to pcbflow
    SkiPart(board.DC(part.pos).right(part.rotate), part, side=part.side)
  except AttributeError:
    print(f"{part.ref} has no pos or side")
    continue

### Preview

In [ ]:
from IPython.display import Image

# Export the current board as a png file
board.save_png("kbd_python", subdir="pcb_png")

# Show
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## Wiring: xiao to switch (ROW)
Route the wires from xiao to ROW.   
Use Turtle syntax to draw wires from xiao and connect them to `Pad` on the switch.

The `Pin` in skidl and `pads` in pcbflow refer to the pins of the component.

In [ ]:
xiaoRef = "U1"

# Function to return Pad number by name from skidl's Net
# skidl's Pin number starts from 1, pcbflow's Pad number starts from 0
def get_pin_number_from_net(netLabel, ref):
    net = Net.get(netLabel)
    return list((int(x.num) - 1 for x in net.pins if x.ref == ref))[0]

# Function to return the connected xiao pad by name from skidl's Net
# Fixes a wrong pad number due to the kicad library and returns it.
# last newpath() fixes strange value in initial value
def xiao_pads(net: str):
  return (
    board.get_part(xiaoRef)
    .pads[get_pin_number_from_net(net, xiaoRef) + 14]
    .newpath()
  )

# Connect the pins of the xiao connected to Net to the pins of the switch
# Pad of xiao on ROWn -> specify layer -> route using Turtle syntax -> align axis up to pad of SW
xiao_pads("ROW0").set_layer(LAYER_TOP).w("r 45").align_meet(
  board.get_part("SW1").pads[0], "x"
)
xiao_pads("ROW1").set_layer(LAYER_TOP).w("r 45").align_meet(
  board.get_part("SW4").pads[0], "y"
)
xiao_pads("ROW2").set_layer(LAYER_TOP).w("l 180 f 12 r 45").align_meet(
  board.get_part("SW8").pads[0], "y"
)

### Preview

In [ ]:
from IPython.display import Image

board.save_png("kbd_python", subdir="pcb_png")

Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## Wiring: diode - switch, switch - switch (ROW), diode - diode (COL)
If the positions of the connected pins are the same, they can be connected in the same code.

In routing COL, a via is created on the way. Store the information of the via and connect it to xiao later.

In [ ]:
# Switches and diodes are arranged in the same way as a block so they can be routed with the same code
# Starts from 1 because of accessing by REF No.
for i in range(1, KEY_COUNT + 1):
  sw = board.get_part(f"SW{i}")
  d = board.get_part(f"D{i}")
  # Specify pad to start routing → Specify layer → Routing in Turtle syntax → Align pad and axis and connect.
  d.pads[0].set_layer(LAYER_BOTTOM).w("f 1 r 45").align_meet(sw.pads[1], "x")

# The pins of the switches on the ROW of the switch matrix are aligned in a straight line, so they can be wired in the same way.
# Note that the number of switches differs depending on the ROW.
for i in range(3):
  # Make an array of Ref No. of connected to SW from Net information of ROW.
  refs = list(x.ref for x in netRows[i].pins if x.ref.startswith("SW"))
  for j in range(len(refs) - 1):
    # Specify pad to start routing → Create new path → Specify layer → Routing in Turtle syntax → Route in a straight line to the next pad
    board.get_part(refs[j]).pads[0].newpath().set_layer(LAYER_TOP).w(
      "r 90 f 2 l 45 f 1 r 45 f 2.5 r 45 f1"
    ).meet(board.get_part(refs[j + 1]).pads[0])

# Array to store via information
# Used to connect with xiao
viaCol = [0] * 4

# The diodes leading to COL1-3 in the switch matrix are aligned in a straight line, so they can be wired in the same way
# Put a via across on the way
for i in range(1, 4):
  d1 = board.get_part(f"D{i}")
  d2 = board.get_part(f"D{i+3}")
  d3 = board.get_part(f"D{i+7}")
  # Diode-Via is routed first to store via information
  via = (
    d1.pads[1]
    .set_layer(LAYER_BOTTOM)
    .w("l 180 f 1.25 r 45 f 2 l 45 f 6.5 l 45")
    .align(d2.pads[1], "y")
    .wire()
    .via()
  )
  viaCol[i] = via
  # Route from via to pad of next diode
  via.set_layer(LAYER_BOTTOM).meet(d2.pads[1])
  d2.pads[1].set_layer(LAYER_BOTTOM).w(
    "l 180 f 1 r 45 f 2 l 45 f 6.5 l 45"
  ).align_meet(d3.pads[1], "y")

### Preview

In [ ]:
from IPython.display import Image

board.save_png("kbd_python", subdir="pcb_png")

Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## Wiring: via to xiao (COL)
Route from the stored via to xiao.

In [ ]:
viaCol[1].set_layer(LAYER_TOP).w("l 45 f 11 l 45").align_meet(
  xiao_pads("COL1"), "x"
)
viaCol[2].set_layer(LAYER_TOP).w("f 2 l 45 f 29.5 l 45").align_meet(
  xiao_pads("COL2"), "x"
)
viaCol[3].set_layer(LAYER_TOP).w("f 4 l 45 f 48.5 l 45").align_meet(
  xiao_pads("COL3"), "x"
)

### Preview

In [ ]:
from IPython.display import Image

board.save_png("kbd_python", subdir="pcb_png")

Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## Wiring: xiao to OLED
Route from xiao to OLED.

Enumerate the name of Net and look up the number of the pad connected from skidl's Net information.

In [ ]:
oledRef = "DISP1"

for net in ["GND", "3.3V", "SCL", "SDA"]:
  oledPin = board.get_part(oledRef).pads[get_pin_number_from_net(net, oledRef)]
  xiao_pads(net).set_layer(LAYER_TOP).left(135).align_meet(oledPin, "y")

## Routing is completed
Now all the routing is done!

In [ ]:
from IPython.display import Image

board.save_png("kbd_python", subdir="pcb_png")

Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## Place pictures on PCB
Place a logo and other images on the silk layer of the PCB.
The image data to be used is in the resource at the beginning.

In [ ]:
DIR_IMGS = "keyboard-made-by-python/"

board.add_bitmap(
  (47.5, 28.5), 
  DIR_IMGS + "hardware/imgs/python-logo.png",
  side="top",
  scale=0.85
)

board.add_bitmap(
  (11.25, 45),
  DIR_IMGS + "hardware/imgs/QR.png",
  side="bottom",
  scale=0.75,
)

board.add_bitmap(
  (11, 28.5),
  DIR_IMGS + "hardware/imgs/logo-python-powered-w-logo.png",
  side="bottom",
  layer="GBS",
  scale=0.7,
)
board.add_bitmap(
  (11, 28.5),
  DIR_IMGS + "hardware/imgs/logo-python-powered-w-logoBG.png",
  side="bottom",
  layer="GBL",
  scale=0.7,
)
board.add_bitmap(
  (11, 28.5),
  DIR_IMGS + "hardware/imgs/logo-python-powered-w-text.png",
  side="bottom",
  scale=0.7,
)

board.add_bitmap(
  (47.5, 22),
  DIR_IMGS + "hardware/imgs/dm9-logo.png",
  side="bottom",
  scale=0.8,
)

board.add_bitmap(
  (66.5, 22),
  DIR_IMGS + "hardware/imgs/hsgw-logo.png",
  side="bottom",
  scale=0.4,
)

### Preview

In [ ]:
from IPython.display import Image

board.save_png("kbd_python", subdir="pcb_png")

Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## Place text on PCB
Place texts on the silk layer of the PCB.

Specify the position where the order number will be placed because I will order to JLCPCB.

In [ ]:
board.add_text(
  (47.5, 41.5),
  "https://github.com/hsgw/keyboard_made_by_python",
  scale=1.25,
  side="bottom",
)

board.add_text(
  (BOARD_WIDTH - 39, 5),
  "KEYBOARD MADE BY PYTHON",
  scale=2,
  side="bottom",
  justify="left",
)
board.add_text(
  (BOARD_WIDTH - 44, 4.75),
  "Rev.1",
  scale=1.25,
  side="bottom",
  justify="left",
)
board.add_text(
  (BOARD_WIDTH - 47.5, 2.25),
  "(c) 2022, Takuya Urakawa / Dm9Records / 5z6p.com",
  scale=1.25,
  side="bottom",
  justify="left",
)

# MAGIC WORD for JLCPCB
board.add_text(
  (11.5, 55),
  "JLCJLCJLCJLC",
  scale=1.1,
  side="bottom",
)

### Preview

In [ ]:
from IPython.display import Image

board.save_png("kbd_python", subdir="pcb_png")

Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

# Complete PCB and generate gerber files
Finally, export the Gerber file for manufacturing and complete PCB.

In [ ]:
board.save_gerbers("kbd_python", subdir="pcb_gerber")

Use kicad's Gerber Viewer to review Gerber files.

![kicad's gerber viewer](../imgs/kicad_gerberviewer.png)

It is also possible to convert from kicad's gerber viewer to pcbnew. For ordering, I converted to pcbnew, modified the silk and generated the gerber file again.

![PCB converted to pcbnew](../imgs/kicad_gerber_to_pcbnew.png)

# Order PCB

! [PCB received](. /imgs/pcb.jpg)

I ordered a batch of Gerber files to JLCPCB, about $8 for 5 boards, and they arrived in less than 2 weeks.

# Soldering the components

![soldered PCB](../imgs/soldered_pcb_top.jpg)
![soldered PCB](../imgs/soldered_pcb_bottom.jpg)

I soldered the components. The PCB design is now complete.